In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

# Load and split data into training and testing sets
df = spark.read.csv("/FileStore/tables/Iris_modified6_5mod.csv", header = "true", sep=",", inferSchema=True)
df.show()

+-----+---+---+---+---+---+---+
label| a0| a1| a2| a3| a4| a5|
+-----+---+---+---+---+---+---+
 1| 1| 32|5.4|3.4|1.5|0.4|
 1| 2| 41|5.0|3.5|1.3|0.3|
 0| 3| 55|6.5|2.8|4.6|1.5|
 0| 4| 71|5.9|3.2|4.8|1.8|
 0| 5| 79|6.0|2.9|4.5|1.5|
 0| 6| 87|6.7|3.1|4.7|1.5|
 0| 7| 93|5.8|2.6|4.0|1.2|
 1| 8| 1|5.1|3.5|1.4|0.2|
 1| 9| 9|4.4|2.9|1.4|0.2|
 1| 10| 14|4.3|3.0|1.1|0.1|
 1| 11| 40|5.1|3.4|1.5|0.2|
 0| 12| 60|5.2|2.7|3.9|1.4|
 0| 13| 95|5.6|2.7|4.2|1.3|
 1| 14| 17|5.4|3.9|1.3|0.4|
 1| 15| 20|5.1|3.8|1.5|0.3|
 1| 16| 24|5.1|3.3|1.7|0.5|
 1| 17| 28|5.2|3.5|1.5|0.2|
 1| 18| 36|5.0|3.2|1.2|0.2|
 1| 19| 44|5.0|3.5|1.6|0.6|
 1| 20| 47|5.1|3.8|1.6|0.2|
+-----+---+---+---+---+---+---+
only showing top 20 rows

In [2]:
# Rename first column to "label"
df = df.withColumnRenamed(df.columns[0], 'label')

# Split into training and test data
training, testing = df.randomSplit([0.6, 0.4], seed=42)

#print "Training"
training.show()


+-----+---+---+---+---+---+---+
label| a0| a1| a2| a3| a4| a5|
+-----+---+---+---+---+---+---+
 0| 7| 93|5.8|2.6|4.0|1.2|
 0| 12| 60|5.2|2.7|3.9|1.4|
 0| 13| 95|5.6|2.7|4.2|1.3|
 0| 22| 59|6.6|2.9|4.6|1.3|
 0| 23| 70|5.6|2.5|3.9|1.1|
 0| 25| 82|5.5|2.4|3.7|1.0|
 0| 26| 85|5.4|3.0|4.5|1.5|
 0| 27| 89|5.6|3.0|4.1|1.3|
 0| 33| 53|6.9|3.1|4.9|1.5|
 0| 34| 63|6.0|2.2|4.0|1.0|
 0| 37| 97|5.7|2.9|4.2|1.3|
 0| 45| 58|4.9|2.4|3.3|1.0|
 0| 46| 69|6.2|2.2|4.5|1.5|
 0| 47| 74|6.1|2.8|4.7|1.2|
 0| 48| 81|5.5|2.4|3.8|1.1|
 0| 49| 84|6.0|2.7|5.1|1.6|
 0| 53| 62|5.9|3.0|4.2|1.5|
 0| 54| 67|5.6|3.0|4.5|1.5|
 0| 60| 77|6.8|2.8|4.8|1.4|
 0| 67| 57|6.3|3.3|4.7|1.6|
+-----+---+---+---+---+---+---+
only showing top 20 rows

In [3]:
#print "Testing"
testing.show()

+-----+---+---+---+---+---+---+
label| a0| a1| a2| a3| a4| a5|
+-----+---+---+---+---+---+---+
 0| 3| 55|6.5|2.8|4.6|1.5|
 0| 4| 71|5.9|3.2|4.8|1.8|
 0| 5| 79|6.0|2.9|4.5|1.5|
 0| 6| 87|6.7|3.1|4.7|1.5|
 0| 24| 75|6.4|2.9|4.3|1.3|
 0| 28|100|5.7|2.8|4.1|1.3|
 0| 35| 66|6.7|3.1|4.4|1.4|
 0| 36| 91|5.5|2.6|4.4|1.2|
 0| 59| 52|6.4|3.2|4.5|1.5|
 0| 61| 88|6.3|2.3|4.4|1.3|
 0| 62| 94|5.0|2.3|3.3|1.0|
 0| 63| 96|5.7|3.0|4.2|1.2|
 0| 68| 65|5.6|2.9|3.6|1.3|
 0| 70| 78|6.7|3.0|5.0|1.7|
 0| 71| 90|5.5|2.5|4.0|1.3|
 0|101| 64|6.1|2.9|4.7|0.1|
 0|102| 99|5.1|2.5|3.0|0.2|
 1| 2| 41|5.0|3.5|1.3|0.3|
 1| 8| 1|5.1|3.5|1.4|0.2|
 1| 9| 9|4.4|2.9|1.4|0.2|
+-----+---+---+---+---+---+---+
only showing top 20 rows

In [4]:
# Configure an ML pipeline, which consists of two stages: feature assembler and lr.
# Transform n feature vectors into one single vector column
# Don't use the first three columns as features since it will result in a prediction accuracy of 100%
#assembler = VectorAssembler(inputCols=training.columns[3:], outputCol='features')

# Test with column 5 to see that some entries are classified wrongly
assembler = VectorAssembler(inputCols=training.columns[5:], outputCol='features')

lr = LogisticRegression(maxIter=10, regParam=0.01)
pipeline = Pipeline(stages=[assembler, lr])
#print "LogisticRegression parameters:\n" + lr.explainParams() + "\n"

# predict 
model = pipeline.fit(training)
prediction = model.transform(testing)

#print "prediction-schema: ", prediction.printSchema()
selected = prediction.select("features", "label", "probability", "prediction")
print ("Label vs. prediction", selected.show())

+---------+-----+--------------------+----------+
 features|label| probability|prediction|
+---------+-----+--------------------+----------+
[4.6,1.5]| 0|[0.99491845702985...| 0.0|
[4.8,1.8]| 0|[0.99837590339846...| 0.0|
[4.5,1.5]| 0|[0.99384878230698...| 0.0|
[4.7,1.5]| 0|[0.99580290469472...| 0.0|
[4.3,1.3]| 0|[0.98513993446252...| 0.0|
[4.1,1.3]| 0|[0.97832928702830...| 0.0|
[4.4,1.4]| 0|[0.99043013874804...| 0.0|
[4.4,1.2]| 0|[0.98421700830157...| 0.0|
[4.5,1.5]| 0|[0.99384878230698...| 0.0|
[4.4,1.3]| 0|[0.98770531185225...| 0.0|
[3.3,1.0]| 0|[0.81951829651619...| 0.0|
[4.2,1.2]| 0|[0.97699324497930...| 0.0|
[3.6,1.3]| 0|[0.94528406269604...| 0.0|
[5.0,1.7]| 0|[0.99857491290381...| 0.0|
[4.0,1.3]| 0|[0.97385931051166...| 0.0|
[4.7,0.1]| 0|[0.87246224604569...| 0.0|
[3.0,0.2]| 0|[0.25167975227478...| 1.0|
[1.3,0.3]| 1|[0.01626639049754...| 1.0|
[1.4,0.2]| 1|[0.01531563698339...| 1.0|
[1.4,0.2]| 1|[0.01531563698339...| 1.0|
+---------+-----+--------------------+----------+
only showing top 20 rows

Label vs. prediction None

In [5]:
# Caculate prediction accuracy
numRows = selected.count()
correct = 0.0
for row in selected.collect():
	if row[1] == row[3]:
		correct = correct+1
	else:
		print(row)

accuracy = correct / numRows
print("Accuracy: ", accuracy)

Row(features=DenseVector([3.0, 0.2]), label=0, probability=DenseVector([0.2517, 0.7483]), prediction=1.0)
Accuracy: 0.972972972972973